In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#installs
!pip install stellargraph
!pip install pyyaml h5py
!pip install -q -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 37.0 MB/s 
     |████████████████████████████████| 1.6 MB 47.1 MB/s 


In [ ]:
#imports 

from stellargraph import StellarGraph
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification, DeepGraphCNN
import pandas as pd
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import math
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score, precision_score, confusion_matrix, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
import warnings
from sklearn import metrics
import keras_tuner as kt
warnings.filterwarnings("ignore")

### Data Loading

In [ ]:
combined_phenotype='/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Metadata/phenotypic_data.csv'
aug2_label = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Metadata/aug_labels.csv'
labels_mappings= pd.read_csv(combined_phenotype)
aug2_labels = pd.read_csv(aug2_label)

In [ ]:
def row_transform(arr, threshold):
  for i in range(len(arr)):
      arr[i] = arr[i] if arr[i]> threshold else abs(arr[i])+0.0001
  return arr

def binarize(df, threshold):
    df = df.transform(lambda x: row_transform(x, threshold))

In [ ]:
ub_02 = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Binary 0_2'
uw = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Weighted v2'

binary_features_02 = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/UCLA Binary Local 0_2/'
weighted_features = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Weighted local feature files/'

uba = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Augmented v1 Binary 0_2'
uwa = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Augmented v1'

weighted_features_aug = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Local Weighted features - UCLA Aug/'
binary_features_aug = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Local Binary features -  UCLA Aug/'

uba1 = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Augmented v2 Binary 0_2/'
uwa1 = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Preprocessed Data/UCLA Augmented v2/'

weighted_features_aug1 = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Local Weighted features- UCLA Aug v2/'
binary_features_aug1 = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Local Binary features - UCLA Aug v2/'

node2vec_features = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/node2vec/'

In [ ]:
graphs_dir = [uba1, uba, ub_02]
feat_dir = [(binary_features_02, "binary_node_features_ucla_binary_"), (weighted_features, "weighted_node_features_")]
aug_feat_dir = [(binary_features_aug, "binary_node_features_"), (weighted_features_aug, "weighted_node_features_")]
aug_feat_dir1 = [(binary_features_aug1, "binary_node_features_"), (weighted_features_aug1, "weighted_node_features_")]

In [ ]:
graphs = list()
graph_labels = list()

# Graph creation
for directory in graphs_dir:
  for file in os.listdir(directory):
    if '(' in file:
        continue
    if file.endswith('.csv'):
      if (directory == ub_02):
          subject = file[12:16]
          if int(subject) == 2155:
            continue
      elif (directory == uw):
          subject = file[14:18]
      elif (directory == uba):
          subject = file[12:16]
          if int(subject) < 3125:
            continue
      elif (directory == uba1):
          subject = file[12:16]
      else:
        print("INVALID DIRECTORY")
        exit(0)

      # APPENDING LABEL
      subject = int(subject)
      print(subject)
      mask = labels_mappings['Subject'] == subject
      #only for aug v2
      if directory ==  uba1:
        if(labels_mappings[mask]['Label'].values[0]==0 or 
            (labels_mappings[mask]['Label'].values[0]!=aug2_labels.query("Subject==@subject")["assumed"].values[0])):
            continue
      graph_labels.append(labels_mappings[mask]['Label'].values[0])

      # APPENDING CORRESPONDING GRAPH
      df = pd.read_csv(directory+'/'+file, header=None)
      df = df.fillna(0)
      if directory == uba:
        for col in df:
          df[col] = df[col].astype(np.int64)
      G = nx.from_pandas_adjacency(df)          

      # Features
      if directory == uba: 
        if len(feat_dir) == 1:
          node_data = pd.read_csv(aug_feat_dir[0][0]+aug_feat_dir[0][1]+str(subject)+'.csv')
          node_data = node_data.drop(['Unnamed: 0'], axis=1, errors='ignore')
        else: 
          node_data_1 = pd.read_csv(aug_feat_dir[0][0]+aug_feat_dir[0][1]+str(subject)+'.csv')
          node_data_2 = pd.read_csv(aug_feat_dir[1][0]+aug_feat_dir[1][1]+str(subject)+'.csv')
          node_data_1 = node_data_1.drop(['Unnamed: 0'], axis=1, errors='ignore')
          node_data_2 = node_data_2.drop(['Unnamed: 0'], axis=1, errors='ignore')
          node_data = pd.merge(node_data_1, node_data_2, how = "inner", left_index=True, right_index=True)
      elif directory == uba1:
        if len(feat_dir) == 1:
          node_data = pd.read_csv(aug_feat_dir1[0][0]+aug_feat_dir1[0][1]+str(subject)+'.csv')
          node_data = node_data.drop(['Unnamed: 0'], axis=1, errors='ignore')
        else: 
          node_data_1 = pd.read_csv(aug_feat_dir1[0][0]+aug_feat_dir1[0][1]+str(subject)+'.csv')
          node_data_2 = pd.read_csv(aug_feat_dir1[1][0]+aug_feat_dir1[1][1]+str(subject)+'.csv')
          node_data_1 = node_data_1.drop(['Unnamed: 0'], axis=1, errors='ignore')
          node_data_2 = node_data_2.drop(['Unnamed: 0'], axis=1, errors='ignore')
          node_data = pd.merge(node_data_1, node_data_2, how = "inner", left_index=True, right_index=True) 
      else:
        if len(feat_dir) == 1:
          node_data = pd.read_csv(feat_dir[0][0]+feat_dir[0][1]+str(subject)+'.csv')
          node_data = node_data.drop(['Unnamed: 0'], axis=1, errors='ignore')
        else:
          node_data_1 = pd.read_csv(feat_dir[0][0]+feat_dir[0][1]+str(subject)+'.csv')
          node_data_2 = pd.read_csv(feat_dir[1][0]+feat_dir[1][1]+str(subject)+'.csv')
          node_data_1 = node_data_1.drop(['Unnamed: 0'], axis=1, errors='ignore')
          node_data_2 = node_data_2.drop(['Unnamed: 0'], axis=1, errors='ignore')
          node_data = pd.merge(node_data_1, node_data_2, how = "inner", left_index=True, right_index=True)

      n2v_data =  pd.read_csv(node2vec_features+ 'node2vec_features_ucla_binary_'+str(subject)+'.csv')
      n2v_data = n2v_data.drop(['Unnamed: 0'], axis=1, errors='ignore')
      node_data = pd.merge(node_data,n2v_data,  how = "inner", left_index=True, right_index=True)
      node_data = node_data.drop(['subgraph centrality'], axis=1, errors='ignore')

      # Standardisation
      cols = list(node_data)
      scaler = StandardScaler().fit(node_data)
      node_data = scaler.transform(node_data)
      node_data = pd.DataFrame(node_data, columns = cols)
      
      g = StellarGraph.from_networkx(G,node_features=node_data)
      graphs.append(g)

In [ ]:
graph_labels = pd.DataFrame(graph_labels)

In [ ]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

## DGCNN Model with 10 fold cross validation

In [ ]:
generator = PaddedGraphGenerator(graphs=graphs)

In [ ]:
epochs=100

In [ ]:
acc = []
spec = []
sens = []
prec = []
f1 = []

In [1]:
for rs in list(range(1,10)):
    es = EarlyStopping(monitor="val_acc", min_delta=0, patience=30, restore_best_weights=True, )

    k = 20 
    in_feat = 27
    layer_sizes = [in_feat, 32, 64, 128, 128, 64, 32, 16, 4, 2]
    dgcnn_model = DeepGraphCNN(
        layer_sizes=layer_sizes,
        activations=["leaky_relu", "leaky_relu", "leaky_relu",  "leaky_relu", "leaky_relu", "leaky_relu", "leaky_relu", "leaky_relu", "leaky_relu", "log_softmax"],
        k=k,
        bias=True,
        generator=generator,
    )
    x_inp, x_out = dgcnn_model.in_out_tensors()

    x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
    x_out = MaxPool1D(pool_size=2)(x_out)
    x_out = Conv1D(filters=32, kernel_size=4, strides=1)(x_out)
    x_out = Flatten()(x_out)
    x_out = Dense(units=128, activation="leaky_relu")(x_out)
    x_out = Dropout(rate=0.5)(x_out)

    predictions = Dense(units=1, activation="sigmoid")(x_out)

    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss=binary_crossentropy, metrics=["acc"],)

    train_graphs, test_graphs = model_selection.train_test_split(graph_labels, train_size=0.8, test_size=0.2, stratify=graph_labels, random_state=rs)
    gen = PaddedGraphGenerator(graphs=graphs)

    train_gen = gen.flow(
        list(train_graphs.index - 1),
        targets=train_graphs.values,
        batch_size=50,
        symmetric_normalization=True,
    )

    test_gen = gen.flow(
        list(test_graphs.index - 1),
        targets=test_graphs.values,
        batch_size=1,
        symmetric_normalization=True,
    )

    history = model.fit(train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True, callbacks=[es],)
    test_metrics = model.evaluate(test_gen)
    pred = model.predict(test_gen)
    y_pred = []
    y_true = []
    x = np.array(pred)
    for i in x:
        if i <0.5:
            y_pred.append(0)
        else:
            y_pred.append(1)

    for t in test_gen.targets:
        y_true.append(t[0])

    accuracy = metrics.accuracy_score(y_true, y_pred)
    f1_score = metrics.f1_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall =  metrics.recall_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)

    acc.append(accuracy)
    spec.append(specificity)
    sens.append(recall)
    prec.append(precision)
    f1.append(f1_score)

In [ ]:
print("accuracy")
print(np.mean(acc), " +- ", np.std(acc))
print()

print("specificity")
print(np.mean(spec), " +- ", np.std(spec))
print()

print("sensitivity:")
print(np.mean(sens), " +- ", np.std(sens))
print()

print("precision")
print(np.mean(prec), " +- ", np.std(prec))
print()

print("f1")
print(np.mean(f1), " +- ", np.std(f1))
print()

accuracy
0.8049999999999999  +-  0.021794494717703346

specificity
0.745  +-  0.05634713834792323

sensitivity:
0.865  +-  0.03427827300200523

precision
0.7746346911029892  +-  0.0350793599648721

f1
0.8161985673306428  +-  0.017452551539528235

